In [ ]:
import numpy as np
import plotly.graph_objects as go

from duqling_py import Duqling

In [ ]:
duqling = Duqling()

Given some integer $n$ and duqling function $f$ with input dimension $m$:

1. Denote $n$ equally spaced points along some dimension by the vector $\vec{v} = [\underbrace{\,0\,,\,\cdots,\,1\,}_n]$

    Let $\mathscr{S}$ be the set of $m$ vectors $\vec{v}$ (essentially representing $n$ equally spaced points along each input dimension)
    $$
    \mathscr{S} = \{\underbrace{\vec{v}, \ldots, \vec{v}}_m\}
    $$

2. Obtain all possible pairs of points $A = \mathscr{S} \times \mathscr{S}$

In [ ]:
def quack_grid_data(function_name, n_points=50, **kwargs):
    
    func_info = duqling.quack(function_name)
    input_dim = int(func_info['input_dim'])

    # n equally spaced points along each input dim
    axes = [np.linspace(0, 1, n_points) for _ in range(input_dim)]
    
    # the cartesian product of these points
    mesh = np.meshgrid(*axes, indexing='ij')
    X_grid = np.stack([m.ravel() for m in mesh], axis=-1)
    
    y = duqling.duq(X=X_grid, f=function_name, **kwargs)

    return X_grid, y

To build a 3D mesh, we first need to obtain a grid of points, which is done by 

1. aggregating the second dimension of $X$ into two seperate arrays which are reshaped to be square matrices: 
$\displaystyle 
\underset{(n^2,\,2)}{X} 
\longrightarrow 
\{
\underset{(n,\,n)}{X'_1},
\underset{(n,\,n)}{X'_2}
\}
$

2. reshaping $y$ similarly:
$
\underset{(n^2,\,1)}{y}
\longrightarrow 
\underset{(n,\,n)}{y}
$

> Recall: $n$ is the number of points sampled

In [ ]:
# note: the response type specification doesn't actually
#       filter any data out, it's just for clarity.
for func_name in duqling.quack(input_dim=2, response_type='uni').fname:
    X, y = quack_grid_data(func_name, n_points=100)

    x1_unique = np.unique(X[:, 0])
    x2_unique = np.unique(X[:, 1])
    nx, ny = len(x1_unique), len(x2_unique)
    x1_grid = X[:, 0].reshape(nx, ny)
    x2_grid = X[:, 1].reshape(nx, ny)
    y_grid = y.reshape(nx, ny)

    fig = go.Figure(go.Surface(x=x1_grid, y=x2_grid, z=y_grid))
    fig.update_layout(dict(title=func_name, height=800))
    fig.show()